In [1]:
!pwd

/Users/miles/proj/tripshark/nb


In [2]:
!wc ../data/1_gtfs/routes.txt

     227    1363   23818 ../data/1_gtfs/routes.txt


In [4]:
import pandas as pd

In [14]:
import os

In [15]:
gtfs_files = os.listdir('../data/1_gtfs')

In [26]:
gtfs = {filename.split('.')[0]: pd.read_csv(
        os.path.join('../data/1_gtfs', filename)) for filename in gtfs_files}

In [27]:
for table in gtfs_tables:
    print(table + '\n' + '-'*len(table))
    print(gtfs[table].head())
    print('\n')

fare_rules
----------
   fare_id  route_id  origin_id  destination_id
0      101  100001.0        NaN             NaN
1      110  100001.0        NaN             NaN
2      101  100002.0        NaN             NaN
3      110  100002.0        NaN             NaN
4      102  100003.0        NaN             NaN


agency
------
  agency_id                agency_name  \
0         1              Metro Transit   
1        23            City of Seattle   
2        40              Sound Transit   
3       KMD  Kingcounty Marine Divison   

                                          agency_url      agency_timezone  \
0                        http://metro.kingcounty.gov  America/Los_Angeles   
1                             http://www.seattle.gov  America/Los_Angeles   
2                        http://www.soundtransit.org  America/Los_Angeles   
3  http://www.kingcounty.gov/transportation/kcdot...  America/Los_Angeles   

  agency_lang    agency_phone  \
0          EN    206-553-3000   
1          

In [30]:
gtfs['trips'].shape

(52194, 10)